<a href="https://colab.research.google.com/github/divyaprabhakaran7/Code-Mixed-Spanish-VLM-Study/blob/main/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torchvision

In [ ]:
pip install tqdm

In [12]:
import pandas as pd
import torch
import zipfile
from io import BytesIO
from PIL import Image
from IPython.display import display
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics import classification_report
import ipywidgets as widgets

# Upload widgets
upload_csv = widgets.FileUpload(accept='.csv', multiple=False, description='Upload CSV')
upload_zip = widgets.FileUpload(accept='.zip', multiple=False, description='Upload Images (.zip)')
display(upload_csv, upload_zip)

# Read uploaded CSV
def get_dataframe(upload_widget):
    for file in upload_widget.value.values():
        content = file['content']
        return pd.read_csv(BytesIO(content))
    return None

# Extract images and flatten folder structure
image_dict = {}

def extract_images(upload_widget):
    for file in upload_widget.value.values():
        zf = zipfile.ZipFile(BytesIO(file['content']))
        for full_path in zf.namelist():
            if full_path.lower().endswith(('.jpg', '.jpeg', '.png')) and not full_path.endswith('/'):
                filename = full_path.split('/')[-1]  # remove folder prefix (Positive/Neutral/etc.)
                with zf.open(full_path) as img_file:
                    try:
                        image_dict[filename] = Image.open(img_file).convert("RGB")
                    except:
                        print(f"Could not load image: {filename}")


FileUpload(value={}, accept='.csv', description='Upload CSV')

FileUpload(value={}, accept='.zip', description='Upload Images (.zip)')

In [ ]:
from PIL import Image
import requests
import torch



# Test text and dummy image
text = "A happy tweet"
image = Image.new("RGB", (224, 224), color="white")

inputs = processor(text=[text], images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

print("✅ Model forward pass completed.")


✅ Model forward pass completed.


In [ ]:
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

print("✅ CLIP model and processor loaded successfully.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ CLIP model and processor loaded successfully.
